<a href="https://colab.research.google.com/github/LCaravaggio/NLP/blob/main/notebooks/04b-FeedforwardLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vamos a entrenar un modelo de lenguaje neuronal feed-forward basado en una ventana de contexto fija y embeddings estáticos. Como datos de entrenamiento, vamos a usar recetas de cocina en español.

## Configuración del entorno

In [1]:
!pip install -qU datasets spacy watermark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platfor

In [2]:
%%capture
!python -m spacy download es_core_news_sm

In [3]:
%reload_ext watermark

In [4]:
%watermark -vmp datasets,spacy,torch,numpy,pandas,tqdm

Python implementation: CPython
Python version       : 3.11.12
IPython version      : 7.34.0

datasets: 3.5.0
spacy   : 3.8.5
torch   : 2.6.0+cu124
numpy   : 2.0.2
pandas  : 2.2.2
tqdm    : 4.67.1

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.123+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



Para usar GPU, arriba a la derecha seleccionar "Change runtime type" --> "T4 GPU".

Es un buena idea desarrollar con CPU, y usar GPU para la corrida final, para que Google no nos limite el uso. En esta notebook puede ser útil usar GPU.

In [5]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Dataset

Vamos a usar un [corpus de recetas de SomosNLP](https://huggingface.co/datasets/somosnlp/RecetasDeLaAbuela).

In [6]:
from datasets import load_dataset

dataset = load_dataset("somosnlp/RecetasDeLaAbuela", "version_1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

main.csv:   0%|          | 0.00/40.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20236 [00:00<?, ? examples/s]

In [7]:
# vemos la estructura:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Id', 'Nombre', 'URL', 'Ingredientes', 'Pasos', 'Pais', 'Duracion', 'Categoria', 'Contexto', 'Valoracion y Votos', 'Comensales', 'Tiempo', 'Dificultad', 'Valor nutricional'],
        num_rows: 20236
    })
})


In [8]:
# Conservamos pais = "ESP":
dataset = dataset.filter(lambda x: x["Pais"] == "ESP")

Filter:   0%|          | 0/20236 [00:00<?, ? examples/s]

In [9]:
# vemos un ejemplo al azar:
dataset["train"][300]

{'Id': 1701,
 'Nombre': 'croquetas de pollo asado',
 'URL': 'https://www.recetasgratis.net/receta-de-croquetas-de-pollo-asado-59330.html',
 'Ingredientes': "['500 gramos de Pollo', '1 unidad de Cebolla', '300 mililitros de Aceite de oliva', '600 mililitros de Leche                                                                    (2½ tazas)', '2 cucharadas soperas de Harina', '1 pizca de Sal', '1 pizca de Nuez moscada molida', '1 pizca de Pimienta negra molida', '1 unidad de Huevo', '200 gramos de Pan rallado']",
 'Pasos': "['1 Reunimos todos los ingredientes para preparar las croquetas de pollo asado.', '2 Colocamos el pollo sin piel en una fuente para horno, salpimentamos, echamos un poco de aceite de oliva y lo metemos al horno a 180 ºC durante 25 minutos.También podéis hacer esta receta de croquetas de pollo asado si os ha sobrado carne de haber preparado, por ejemplo, pollo al horno con patatas y cebolla.', '3 Picamos la cebolla de forma fina y la salteamos en la sartén con un po

In [10]:
# A veces los textos son listas no parseadas como tales.
# En tal caso, hacemos un join de la lista.
import re

def preprocess(example):
    """
    """
    if example["Pasos"].startswith("["):
        pasos_list = eval(example["Pasos"].encode('unicode_escape'))
        example["Pasos"] = " ".join(pasos_list)
    # Eliminamos whitespace duplicado:
    example["Pasos"] = re.sub(r'\s+', ' ', example["Pasos"])
    return example

dataset = dataset.map(preprocess)

Map:   0%|          | 0/9381 [00:00<?, ? examples/s]

In [11]:
dataset["train"][300]

{'Id': 1701,
 'Nombre': 'croquetas de pollo asado',
 'URL': 'https://www.recetasgratis.net/receta-de-croquetas-de-pollo-asado-59330.html',
 'Ingredientes': "['500 gramos de Pollo', '1 unidad de Cebolla', '300 mililitros de Aceite de oliva', '600 mililitros de Leche                                                                    (2½ tazas)', '2 cucharadas soperas de Harina', '1 pizca de Sal', '1 pizca de Nuez moscada molida', '1 pizca de Pimienta negra molida', '1 unidad de Huevo', '200 gramos de Pan rallado']",
 'Pasos': '1 Reunimos todos los ingredientes para preparar las croquetas de pollo asado. 2 Colocamos el pollo sin piel en una fuente para horno, salpimentamos, echamos un poco de aceite de oliva y lo metemos al horno a 180 ºC durante 25 minutos.También podéis hacer esta receta de croquetas de pollo asado si os ha sobrado carne de haber preparado, por ejemplo, pollo al horno con patatas y cebolla. 3 Picamos la cebolla de forma fina y la salteamos en la sartén con un poco de ac

Hacemos un partición train/test y achicamos (solo para trabajar mas rapido). Y conservamos solo el texto de las recetas.

In [12]:
dataset = dataset.shuffle(seed=33)

In [13]:
texts_train = dataset["train"].select(range(0, 4_000))["Pasos"]
texts_test = dataset["train"].select(range(4_000, 8_000))["Pasos"]

In [14]:
import textwrap

print(textwrap.fill(texts_train[33], 100))

1 Tritura las galletas hasta hacerlas casi polvo, puedes ayudarte de una licuadora o una
procesadora. Si no dispones de estos utensilios, utiliza una bolsa plástica y un rodillo. 2 Derrite
la mantequilla hasta que quede líquida y agrégala a las galletas trituradas, mezcla bien hasta
formar una pasta uniforme. 3 Traslada la mezcla al molde y extiéndela para hacer la base de la tarta
de queso, puedes ayudarte de una paleta o hacerlo directamente con los dedos. Déjala reposar en el
refrigerador para que se endurezca mientras preparas el relleno. 4 Lleva la nata a fuego lento y
añade, poco a poco, el queso crema y el azúcar, siempre revolviendo con una paleta o batidor. 5 Para
añadir la cuajada, lo mejor es seguir las instrucciones de uso en su empaque. En este caso, la
disolvimos en 25 ml de leche y la agregamos a la mezcla mientras revolvíamos para una mejor
incorporación. 6 Pasados unos 5 minutos luego de añadir la cuajada, retira la mezcla del fuego y
deja reposar unos 2-3 minutos. Sac

## Construcción del vocabulario y tokenización

Vamos a usar el tokenizer para español de `spacy`.

El objetivo es generar una **lista de n-gramas para entrenar la
NN**. e.g con n=4, queremos tuplas de (3 palabras de contexto, 1 target).

Vamos a:

* Considerar como parte del vocabulario todas las palabras que ocurran al menos dos veces.
* Hacer "padding" con BOS y EOS tokens.
* Tokenizar cada documento y convertir a token IDs según el vocab.
* Pasar de tokens a n-gramas y generar una sola lista con todos los samples de entrenamiento.


In [15]:
# tokenizer con reglas de puntacion, contracciones, etc:
import spacy

tokenizer = spacy.load('es_core_news_sm')

In [16]:
# Veamos un ejemplo:
doc = tokenizer(texts_train[0])
print(doc.text)
for i, token in enumerate(doc):
    print(token.text)
    if i > 15:
        break

1 Cocemos el arroz en agua hirviendo con un poco de sal. 2 Picamos los ajos y la cebolla en cuadraditos.Cortamos el tocino en dados. 3 Cortamos las cabezas de la trucha por detrás de las agallas y con un cuchillo afilado, las abrimos de arriba abajo por el vientre. 4 Retiramos las vísceras y todas las membranas negras. 5 Seccionamos la carne a cada lado de la espina sin llegar al dorso. 6 Retiramos las espinas cortando a la altura de la cola. 7 Las lavamos muy bien y secamos con la ayuda de un paño. 8 Salpimentamos por dentro y por fuera.Lavamos las setas y las picamos. 9 Reservamos la mitad. 10 En una sartén con aceite, sofreímos el tocino y la mitad de las setas. 11 Rellenamos las truchas con las setas y el tocino. 12 Cerramos la abertura. 13 Horneamos durante 25 minutos. 14 Rehogamos en mantequilla el resto de las setas. 15 Añadimos el perejil picado, incorporamos el arroz cocido. 16 Vertemos un chorrito de aceite, añadimos el ajo picado, agregamos la cebolla picada y rehogamos dos 

In [17]:
from tqdm import tqdm

def create_vocab(docs: list, min_frec=2) -> tuple:
    """Crea un vocabulario a partir de una lista de docs.
    Returns:
        Dos diccionarios: token2idx (palabra -> índice) y idx2token (índice -> palabra)
    """
    # NOTE esto se puede acelerar paralelizando la tokenizacion con datasets.map()
    # y luego usar e.g. pandas explode().value_counts(). Además, podriamos
    # aprovechar y ya guardar el dataset de train tokenizado.
    str2count = {}
    for doc in tqdm(docs):
        for token in tokenizer(doc):
            token = token.text
            str2count[token] = str2count.get(token, 0) + 1
    # filtrar por min_frec:
    str2count = {token: count for token, count in str2count.items() if count >= min_frec}
    # ordenar de mayor a menor frecuencia:
    str2count = dict(sorted(str2count.items(), key=lambda x: x[1], reverse=True))
    # Mapeamos cada token a un índice distinto
    token2idx = {token: idx for idx, token in enumerate(str2count)}
    # Agregamos "<unk>", "<bos>", "<eos>"  al vocab:
    token2idx["<unk>"] = len(str2count)
    token2idx["<bos>"] = len(str2count) + 1
    token2idx["<eos>"] = len(str2count) + 2
    # "Invertir" el diccionario:
    idx2token = {idx: token for idx, token in enumerate(token2idx)}
    return token2idx, idx2token


token2idx, idx2token = create_vocab(texts_train)

100%|██████████| 4000/4000 [01:57<00:00, 34.07it/s]


In [18]:
print(len(token2idx))
print(token2idx["<unk>"], token2idx["<bos>"], token2idx["<eos>"])

11341
11338 11339 11340


In [19]:
from torch import Tensor

def tokenize(doc: str, ngram_order: int = 4) -> Tensor:
  """Convierte documento a tensor de token IDs.
  Agrega n-1 BOS y 1 EOS tokens (end-of-seq. y beg-of-seq).
  """
  token_ids = [token2idx.get(token.text, token2idx["<unk>"]) for token in tokenizer(doc)]
  # agregamos BOS y EOS tokens:
  token_ids = [token2idx["<bos>"]] * (ngram_order - 1) + token_ids + [token2idx["<eos>"]]
  return torch.tensor(token_ids, dtype=torch.long)

print(texts_train[0])
print(tokenize(texts_train[0])[:20])

1 Cocemos el arroz en agua hirviendo con un poco de sal. 2 Picamos los ajos y la cebolla en cuadraditos.Cortamos el tocino en dados. 3 Cortamos las cabezas de la trucha por detrás de las agallas y con un cuchillo afilado, las abrimos de arriba abajo por el vientre. 4 Retiramos las vísceras y todas las membranas negras. 5 Seccionamos la carne a cada lado de la espina sin llegar al dorso. 6 Retiramos las espinas cortando a la altura de la cola. 7 Las lavamos muy bien y secamos con la ayuda de un paño. 8 Salpimentamos por dentro y por fuera.Lavamos las setas y las picamos. 9 Reservamos la mitad. 10 En una sartén con aceite, sofreímos el tocino y la mitad de las setas. 11 Rellenamos las truchas con las setas y el tocino. 12 Cerramos la abertura. 13 Horneamos durante 25 minutos. 14 Rehogamos en mantequilla el resto de las setas. 15 Añadimos el perejil picado, incorporamos el arroz cocido. 16 Vertemos un chorrito de aceite, añadimos el ajo picado, agregamos la cebolla picada y rehogamos dos 

In [20]:
def doc2ngrams(doc: str, ngram_order: int = 4) -> list:
  """Convierte un documento en tuplas de
  ([ idx_i-context_size, ..., idx_i-1 ], target_idx), donde cada elemento de la tupla
  es un tensor de token IDs.
  """
  token_ids = tokenize(doc, ngram_order=ngram_order)
  ngrams_list = [
      (token_ids[(i-ngram_order):(i-1)], token_ids[i-1])
      for i in range(ngram_order, len(token_ids) + 1)
  ]
  return ngrams_list

In [21]:
# por ejemplo:
doc_ = texts_train[0]
token_ids_ = tokenize(doc_)
ngrams_ = doc2ngrams(doc_)

print(doc_)
print(token_ids_[:10])
print(ngrams_)

1 Cocemos el arroz en agua hirviendo con un poco de sal. 2 Picamos los ajos y la cebolla en cuadraditos.Cortamos el tocino en dados. 3 Cortamos las cabezas de la trucha por detrás de las agallas y con un cuchillo afilado, las abrimos de arriba abajo por el vientre. 4 Retiramos las vísceras y todas las membranas negras. 5 Seccionamos la carne a cada lado de la espina sin llegar al dorso. 6 Retiramos las espinas cortando a la altura de la cola. 7 Las lavamos muy bien y secamos con la ayuda de un paño. 8 Salpimentamos por dentro y por fuera.Lavamos las setas y las picamos. 9 Reservamos la mitad. 10 En una sartén con aceite, sofreímos el tocino y la mitad de las setas. 11 Rellenamos las truchas con las setas y el tocino. 12 Cerramos la abertura. 13 Horneamos durante 25 minutos. 14 Rehogamos en mantequilla el resto de las setas. 15 Añadimos el perejil picado, incorporamos el arroz cocido. 16 Vertemos un chorrito de aceite, añadimos el ajo picado, agregamos la cebolla picada y rehogamos dos 

In [22]:
# armamos todos los ngrams de training:
ngrams_train = []
for doc in tqdm(texts_train):
  ngrams_train.extend(doc2ngrams(doc, ngram_order=4))

100%|██████████| 4000/4000 [02:08<00:00, 31.24it/s]


In [23]:
print(ngrams_train[:2])

[(tensor([11339, 11339, 11339]), tensor(21)), (tensor([11339, 11339,    21]), tensor(1318))]


## Armado de _batches_

Armamos los batches para entrenar el modelo. Para esto usamos la clase `DataLoader` de PyTorch. En cada iteración, el `DataLoader` nos devuelve un batch de ejemplos. No necesitamos una _collate function_ porque ya todos los ejemplos tienen igual dimensión (no necesitamos padding).

In [24]:
from torch.utils.data import DataLoader

batch_size = 32

train_loader = DataLoader(ngrams_train, batch_size=batch_size, shuffle=True)

In [25]:
# Veamos los primeros dos batches de entrenamiento:
torch.manual_seed(33)
for i, data in enumerate(train_loader):
    print(f"### batch {i}")
    print(f"Shapes = {[s.shape for s in data]}")
    print("Primeros 5 ejemplos:")
    print("- Features:")
    print(data[0][:5])
    print("- Targets:")
    print(data[1][:5])
    print()
    if i == 1:
        break

### batch 0
Shapes = [torch.Size([32, 3]), torch.Size([32])]
Primeros 5 ejemplos:
- Features:
tensor([[  16,   12, 1805],
        [  39, 1214,    6],
        [ 457,    8,    5],
        [  61,    5,   33],
        [   0,   60,   10]])
- Targets:
tensor([  2, 133,  51, 321, 120])

### batch 1
Shapes = [torch.Size([32, 3]), torch.Size([32])]
Primeros 5 ejemplos:
- Features:
tensor([[ 88,   0,  72],
        [ 77,   9,  10],
        [  3,   4, 291],
        [  2,  84,  15],
        [499,   2,  16]])
- Targets:
tensor([   46,   120,     3, 11338,   154])



**PREGUNTA**: ¿Qué información tiene cada ejemplo en un batch?

## Modelo

Armamos una red bien sencilla con una hidden layer. Es la misma arquitectura que Figure 7.17 de [Jurafksy](https://web.stanford.edu/~jurafsky/slp3/). Usamos embeddings con inicialización random pero podríamos empezar con embeddings pre-entrenados.

NOTE: Como vamos a usar [Cross Entropy Loss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html), no tenemos que aplicar softmax porque espera "raw, unnormalized scores for each class" i.e. logits.

In [26]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class NGramLanguageModel(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_size, ngram_order):
        super().__init__()
        context_size = ngram_order - 1
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, hidden_size)
        self.linear2 = nn.Linear(hidden_size, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs) # shape (bsz, context_size, embed_dim)
        concatenated_embeds = embeds.flatten(1) # shape (bsz, context_size * embed_dim)
        hidden = F.relu(self.linear1(concatenated_embeds)) # shape (bsz, hidden_size)
        logits = self.linear2(hidden) # shape (bsz, vocab_size)
        return logits


**PREGUNTA**: ¿en qué atributos de NGramLanguageModel están los pesos de la red? ¿Qué representa el método forward?

## Entrenamiento


In [27]:
# Instanciamos el modelo
neural_lm = NGramLanguageModel(
    vocab_size=len(token2idx),
    embedding_dim=50,
    hidden_size=32,
    ngram_order=4,
)
neural_lm = neural_lm.to(device)

In [28]:
# Funcion de pérdida y optimizador
from torch import optim

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(neural_lm.parameters(), lr=1e-3)

In [29]:
# Loop de entrenamiento (sin datos de validación)

def train_epoch(model, optimizer, train_loader, log_steps=2000, device=None):
    """Entrena 1 epoch
    """
    total_loss = 0
    steps_done = 0
    n_steps = len(train_loader)
    for context, target in tqdm(train_loader, total=n_steps):
        context = context.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        logits = model(context)
        loss = loss_fn(logits, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        steps_done += 1
        train_loss = total_loss / steps_done
        if steps_done % log_steps == 0:
            print(f"    [steps={steps_done}] train_loss: {train_loss:.4f}")
    return train_loss

def train(
    model, optimizer, train_loader, n_epochs, device=None):
  """Entrena el modelo durante n_epochs.
  """
  for epoch in range(n_epochs):
      print(f"Epoch {epoch} / {n_epochs}")
      epoch_loss = train_epoch(model, optimizer, train_loader, device=device)
      print(f"Training loss = {epoch_loss:.3f}")

**PREGUNTA**: ¿Qué es un epoch? ¿Qué es un paso de optimización? ¿Cómo podemos calcular la cantidad total de pasos de optimización?

In [30]:
# entrenamos!
num_epochs = 1
train(neural_lm, optimizer, train_loader, num_epochs, device=device)

Epoch 0 / 1


  7%|▋         | 2083/30585 [00:04<00:59, 480.96it/s]

    [steps=2000] train_loss: 6.0745


 13%|█▎        | 4096/30585 [00:08<00:47, 560.51it/s]

    [steps=4000] train_loss: 5.7148


...

    [steps=28000] train_loss: 4.9031


 98%|█████████▊| 30107/30585 [00:56<00:00, 544.07it/s]

    [steps=30000] train_loss: 4.8829


100%|██████████| 30585/30585 [00:57<00:00, 528.92it/s]

Training loss = 4.877


## Generación de texto

In [31]:
def text2input(context_str: str, ngram_order: int = 4) -> Tensor:
    """Convierte contexto en un input para la NN (tensor de input IDs)
    """
    ngrams = doc2ngrams(context_str, ngram_order=ngram_order)
    # el input es el "contexto" del ultimo ngram
    last_context = ngrams[-1][0]
    # agregamos una dimension que hace las veces de batch (size=1) para hacer el forward
    out = last_context.unsqueeze(0)
    return out

In [32]:
# Ejemplo:
print(text2input("usamos la", ngram_order=4))
print(text2input("", ngram_order=4))

tensor([[11339,  3411,     4]])
tensor([[11339, 11339, 11339]])


In [33]:
def sample_text(model, start_text, max_length=10, ngram_order=4, greedy=False):
    """Generación autorregresiva aleatoria de texto sampleando de softmax.
    El modelo debe ser consistente con ngram_order.
    """
    # buscamos los input IDs segun el context size
    input_ = text2input(start_text, ngram_order=ngram_order)
    # mandamos inputs al mismo device que el modelo
    device = next(model.parameters()).device
    input_ = input_.to(device)
    idx_eos = token2idx["<eos>"]
    context_size = ngram_order - 1
    # el resultado solo va a incluir el contexto usado + el texto nuevo
    idxs_result = input_.clone()
    with torch.inference_mode():
        for i in range(max_length):
            logits = model(input_) # logits
            probas = F.softmax(logits, dim=1) # probas
            if greedy:
                sampled_idx = torch.argmax(probas, dim=1).unsqueeze(1)
            else:
                # sample:
                sampled_idx = torch.multinomial(probas, num_samples=1)
            # actualizamos el resultado
            idxs_result = torch.cat((idxs_result, sampled_idx), dim=1)
            # actualizamos el input conservando solo los ultimos context_size tokens
            input_ = idxs_result[:,-context_size:]
            if sampled_idx == idx_eos:
                break
        tokens_result = [idx2token[idx.item()] for idx in idxs_result.squeeze()]
        return tokens_result

In [34]:
print(texts_test[0])

1 Cortar los calamares en tiras delgadas y se ponen en remojo unas 3 horas con la leche. 2 En un recipiente hondo, se pone la harina, una cucharada de aceite, sal y un dl. de agua. 3 Con todo esto se forma una masa espesa. 4 Se ponen los calamares en la masa, se escurren y se fríen en abundante aceite caliente. 5 Se sirve con unos gajos de limón. 6 Para 4 personas.


In [35]:
torch.manual_seed(0)
start_text = "1 Cortar los calamares"
res_ = sample_text(neural_lm, start_text, ngram_order=4, max_length=50)

print(" ".join(res_))

Cortar los calamares otras . lista en la bandeja para que los trozos , pon a fuego , se pone , colamos EL intacto la mantequilla y las croquetas y ¡ minutos más , vacía y retiramos a 200 ºC : más verde , a doren por encima de este tamaño de manzana


In [36]:
torch.manual_seed(22)
start_text = ""
res_ = sample_text(neural_lm, start_text, ngram_order=4, max_length=50)

print(" ".join(res_))

<bos> <bos> <bos> 1 Para los bordes poniendo este caso de su salsa de coco caldoso , se le picado . Mezcla con la base de quinoa . <eos>


**PREGUNTA**: ¿por qué los textos generados son incoherentes?

In [37]:
start_text = ""
res_ = sample_text(neural_lm, start_text, ngram_order=4, max_length=50, greedy=True)

print(" ".join(res_))

<bos> <bos> <bos> 1 Para empezar a preparar la salsa de la receta de la salsa de la receta de la salsa de la receta de la salsa de la receta de la salsa de la receta de la salsa de la receta de la salsa de la receta de la salsa de


**PREGUNTA**: ¿por qué el texto generado es repetitivo?

## Evaluación

Computamos perplexity (PPL) en test.

* Hacemos $ \exp(\log PPL ) $ para evitar underflow.
* Vean que $\log PPL = CrossEntropy = -avg(\log(probas))$

In [38]:
ngrams_test = []
for doc in tqdm(texts_test):
  ngrams_test.extend(doc2ngrams(doc, ngram_order=4))

100%|██████████| 4000/4000 [02:02<00:00, 32.61it/s]


In [39]:
test_loader = DataLoader(ngrams_test, batch_size=32, shuffle=False)

In [40]:
def perplexity(model, dataloader, device):
    with torch.no_grad():
        # Iteramos por batch. Vamos a ir guardando las probas de los tokens correctos en cada batch.
        all_log_probs_gt = torch.tensor([], device=device) # gt: ground truth
        for context, target in dataloader:
            context = context.to(device)
            target = target.to(device)
            batch_size = len(target)
            logits = model(context) # shape (bsz, vocab_size)
            log_probs = F.log_softmax(logits, dim=1) # shape (bsz, vocab_size)
            # log_probs_gt:
            log_probs_gt = log_probs[torch.arange(batch_size), target] # shape (bsz)
            all_log_probs_gt = torch.cat((all_log_probs_gt, log_probs_gt))
        # Calculamos PPL:
        ce = -all_log_probs_gt.mean()
        res = torch.exp(ce)
    return res.item()

In [41]:
test_ppl = perplexity(neural_lm, test_loader, device)
print(f"Test PPL = {test_ppl:.3f}")

Test PPL = 90.643


**PREGUNTA**: ¿Por qué este modelo tiene mejor rendimiento que el ngram de la notebook "ngramLM"?